In [ ]:
%load_ext autoreload
%autoreload 2

import os
from PIL import Image
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
import dit_fast_attention
import os
file_path=os.path.abspath(dit_fast_attention.__file__)
dir_path=os.path.dirname(file_path)
os.chdir(dir_path)
import matplotlib
import cv2
import numpy as np

In [ ]:
s=["full_attn+cfg_attn_share","residual_window_attn","residual_window_attn+cfg_attn_share","output_share"][::-1]
print(s)

In [ ]:
import torch
from utils import calculate_fvd
from tmp.common_metrics_on_video_quality.calculate_psnr import calculate_psnr
from tmp.common_metrics_on_video_quality.calculate_ssim import calculate_ssim
from tmp.common_metrics_on_video_quality.calculate_lpips import calculate_lpips
NUMBER_OF_VIDEOS = 8
VIDEO_LENGTH = 30
CHANNEL = 3
SIZE = 64
# videos1 = torch.zeros(NUMBER_OF_VIDEOS, VIDEO_LENGTH, CHANNEL, SIZE, SIZE, requires_grad=False)
# videos2 = torch.ones(NUMBER_OF_VIDEOS, VIDEO_LENGTH, CHANNEL, SIZE, SIZE, requires_grad=False)
device = torch.device("cuda")
# device = torch.device("cpu")

def load_to_videos(file_path):
    all_videos=[]
    for file_name in os.listdir(file_path):
        if file_name.endswith('.mp4'):
            print(file_name)
            video = cv2.VideoCapture(file_path+file_name)
            frames = []
            while True:
                ret, frame = video.read()
                if not ret:
                    break
                frames.append(frame)
            video.release()
            # BGR to RGB
            frames = [cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) for frame in frames]
            frames = np.array(frames)
            frames = torch.tensor(frames, dtype=torch.float32).permute(0, 3, 1, 2)/255
            frames = frames.unsqueeze(0)
            all_videos.append(frames)
    all_videos = torch.cat(all_videos, dim=0)
    return all_videos
            
videos1 = load_to_videos('output/opensora/_50_100_1.0_32_(240, 426)/')

for threshold in [0.975,0.95,0.925,0.9,0.875,0.85]:
    videos2=load_to_videos(f'output/opensora/_50_100_{threshold}_32_(240, 426)/')
    # rst=calculate_fvd(raw_videos1, videos2, device, method='styleganv')
    result={}
    # result['ssim'] = calculate_ssim(videos1, videos2)
    # result['psnr'] = calculate_psnr(videos1, videos2)
    result['lpips'] = calculate_lpips(videos1, videos2, device)
    print(result)
    print(result['lpips'])

In [ ]:
import cv2
import numpy as np
import cv2

def extract_and_stitch_frames(video_files, frame_indexes, output_file):
   """
   从视频文件中提取指定帧，并将它们横向拼接在一起。
   多个视频文件的结果按照纵向拼接。
   
   :param video_files: 视频文件的列表。
   :param frame_indexes: 要提取的帧的索引。
   :param output_file: 输出拼接后的图像文件路径。
   """
   all_frames = []
   
   for video_file in video_files:
       cap = cv2.VideoCapture(video_file)
       frames = []
       
       for idx in frame_indexes:
           cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
           ret, frame = cap.read()
           if ret:
               frames.append(frame)
           else:
               print(f"无法从 {video_file} 中读取第 {idx} 帧")
       
       cap.release()
       
       # 将同一视频文件的所有帧横向拼接
       stitched_frame = cv2.hconcat(frames)
       all_frames.append(stitched_frame)
   
   # 将所有视频文件的拼接后的帧纵向拼接
   stitched_image = cv2.hconcat(all_frames)
   
   cv2.imwrite(output_file, stitched_image)
   print(f"已将拼接后的图像保存到 {output_file}")


# 示例用法
for threshold in [1.0,0.975,0.95,0.925,0.9,0.875,0.85]:
    video_files=[]
    frames=[0,7,13]
    for samplei in [2,4,6]:
        video_file=f'output/opensora/_50_100_{threshold}_32_(240, 426)/_{samplei}.mp4'
        video_files.append(video_file)
    output_file = f'plot/stitched_frame_{threshold}.jpg'

    extract_and_stitch_frames(video_files, frames, output_file)

def add_caption_and_stack_images(images, output_file, font=cv2.FONT_HERSHEY_SIMPLEX):
    
    # 读取第一张图片，用于获取字体大小
    image = cv2.imread(images[0])
    height, width, _ = image.shape
    
    # 计算标题字体大小
    font_scale = 0.5
    font_thickness = 1
    text = os.path.splitext(os.path.basename(images[0]))[0]  # 获取文件名作为标题
    (text_width, text_height), _ = cv2.getTextSize(text, font, font_scale, font_thickness)
    
    # 确保图片宽度足够容纳标题
    
    if width < text_width + 10:
        new_width = text_width + 10
        new_height = int(height * (new_width / width))  # 保持图片的长宽比
        image = cv2.resize(image, (new_width, new_height))
    else:
        new_width = width
        new_height = height
    
    # 为图片添加标题
    captioned_image = image.copy()
    cv2.putText(captioned_image, text, (10, int(0.9 * new_height)), font, font_scale, (0, 0, 0), font_thickness)
    
    # 纵向拼接图片
    total_images = len(images)
    stacked_images = []
    for img_path in images:
        img = cv2.imread(img_path)
        height, width, _ = img.shape
        # 调整图片大小以匹配第一张图片的宽度
        resized_img = cv2.resize(img, (new_width, int(height * (new_width / width))))
        # 添加标题
        captioned_img = resized_img.copy()
        cv2.putText(captioned_img, os.path.splitext(os.path.basename(img_path))[0], (10, int(0.9 * new_height)), font, font_scale, (0, 0, 0), font_thickness)
        stacked_images.append(captioned_img)
    
    # 计算纵向拼接后的图片高度
    total_height = sum([img.shape[0] for img in stacked_images])
    stacked_image = np.zeros((total_height, new_width, 3), dtype=np.uint8)
    
    # 拼接图片
    y_offset = 0
    for img in stacked_images:
        stacked_image[y_offset:y_offset + img.shape[0], :, :] = img
        y_offset += img.shape[0]
    
    # 保存拼接后的图片
    # stacked_image=cv2.resize(stacked_image, fx=0.5,fy=0.5)
    cv2.imwrite(output_file, stacked_image)


# 使用示例
image_folder = []
for threshold in [1.0,0.95,0.9]:
    image_folder.append(f'plot/stitched_frame_{threshold}.jpg')
output_file = 'plot/stacked_images.jpg'  # 输出图片路径
add_caption_and_stack_images(image_folder, output_file)

In [ ]:
videos2.shape

In [ ]:
from torch import tensor
import argparse
from argparse import Namespace
from collections import namedtuple
import re
def parse_txt_data(txt_data):
    data = []
    for block in txt_data.split('\n\n'):
        lines = block.strip().split('\n')
        for line in lines:
            if line.startswith('Config '):
                idx = line.find("{")
                namespace_dict=eval(line[idx:])
                data.append(namespace_dict)
            # elif line.startswith('calib_ssim'):
            #     calib_ssim = float(line.split('=')[1])
            #     # data.append(calib_ssim)
            #     namespace_dict["calib_ssim"] = calib_ssim
            elif line.startswith('{'):
                metrics = eval(line)
                # data.append(metrics)
                namespace_dict["metrics"] = metrics
            elif line.startswith('macs'):
                macs = eval(line.split('=')[1])
                namespace_dict["macs"] = macs
                # data.append(macs)
            elif line.startswith('attn_mac'):
                attn_mac = float(line.split('=')[1])
                # data.append(attn_mac)
                namespace_dict["attn_mac"] = attn_mac
            elif line.startswith('latencies'):
                latencies = eval(line.split('=')[1])
                # data.append(latencies)
                namespace_dict["latencies"] = latencies
                # print(latencies)
    return data

txt_data=open("output/opensora_results.txt").read()
data=parse_txt_data(txt_data)


In [ ]:
# Ablation plot
seqs=[1,0.975,0.95,0.925,0.9,0.875,0.85]
# sfig, saxes = plt.subplots(1,3,figsize=(3.8*3,1.5))
fig=plt.figure(figsize=(3,2))
ax=fig.add_subplot(111)
# titles=["512x512 DiT-XL","1024x1024 PixArt-Sigma-XL","2048x2048 PixArt-Sigma-XL"]

visited=[]
attn_macs=[]
for threshold in seqs:
    for l in data[::-1]:
        # print(l)
        if l["threshold"]==threshold and l["scheduler"]["num_sampling_steps"]==50 and l["image_size"]==(240,426):
            if threshold in visited:
                continue
            visited.append(l["threshold"])
            attn_macs.append(l["attn_mac"]/1e3)
            # lats_all.append(l["latencies"][f"{nbatch}_all"])
            # lats_attn.append(l["latencies"][f"{nbatch}_attn"])
print(attn_macs)
attn_macs=np.array(attn_macs)
ax.plot(seqs,attn_macs,marker='o',label="Attention MACs (G)")
# print(attn_macs/attn_macs[0]*100)
print_macs_frac=[f"{x:.2f}\%" for x in 100-attn_macs/attn_macs[0]*100]
print(', '.join(print_macs_frac))
    

In [ ]:
import cv2
import numpy as np
import cv2

def extract_and_stitch_frames(video_filess, frame_indexes, output_file):
    """
    从视频文件中提取指定帧，并将它们横向拼接在一起。
    多个视频文件的结果按照纵向拼接。

    :param video_files: 视频文件的列表。
    :param frame_indexes: 要提取的帧的索引。
    :param output_file: 输出拼接后的图像文件路径。
    """
    stitched_images=[]
    for video_files in video_filess:
        all_frames = []

        for video_file in video_files:
            cap = cv2.VideoCapture(video_file)
            frames = []
            
            for idx in frame_indexes:
                cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
                ret, frame = cap.read()
                if ret:
                    frames.append(frame)
                else:
                    print(f"无法从 {video_file} 中读取第 {idx} 帧")
            
            cap.release()
            
            # 将同一视频文件的所有帧横向拼接
            stitched_frame = cv2.vconcat(frames)
            all_frames.append(stitched_frame)

        # 将所有视频文件的拼接后的帧纵向拼接
        stitched_image = cv2.vconcat(all_frames)
        stitched_images.append(stitched_image)
    # all_images=np.array(stitched_images)
    all_images = cv2.hconcat(stitched_images)
    
    all_images=cv2.resize(all_images, None, fx=0.5, fy=0.5)
    cv2.imwrite(output_file, all_images)
    print(f"已将拼接后的图像保存到 {output_file}")


# 示例用法
video_filess=[]
for threshold in [1.0,0.975,0.95,0.925,0.9,0.875,0.85]:
    video_files=[]
    frames=[_ for _ in range(0,16,8)]
    for samplei in range(0,10):
        video_file=f'output/opensora/_50_100_{threshold}_32_(240, 426)/_{samplei}.mp4'
        video_files.append(video_file)
    
    video_filess.append(video_files)
output_file = f'plot/stitched_frame_appendix.jpg'
extract_and_stitch_frames(video_filess, frames, output_file)
